In [1]:
import requests
import json

In [2]:
def getDistances(sourcePs, destPs, metrics = "duration"):
    # bei "duration" antwort in Sekunden
    # dei "distance" antwort in Metern
    url = "https://api.openrouteservice.org/v2/matrix/driving-car"
    N = len(sourcePs)
    M = len(destPs)
    locations = sourcePs + destPs
    body = {"locations":locations, "destinations":list(range(N,N+M)), "sources":list(range(0,N)), "metrics":[metrics]}
    # spalten nach destinations, zeile nach sources
    headers = {'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf6248313acae367214955a3dcb710c5e4e0b8',
    'Content-Type': 'application/json; charset=utf-8'}
    call = requests.post(url, json=body, headers=headers)
    #print(call)
    edgeValues = json.loads(call.text)[metrics+"s"]
    return edgeValues

In [3]:
def createForks(sourcePIDs, destPIDs, distMatrix):
    retDict = {}
    for sPID in sourcePIDs:
        retDict[sPID] = {}
    for j in range(0, len(sourcePIDs)):
        sourcePID = sourcePIDs[j]
        for i in range(0, len(destPIDs)):
            #print(i)
            retDict[sourcePID][destPIDs[i]] = distMatrix[j][i]
    return retDict

In [4]:
def createDistGraph(pDict, forkDict, metrics = "duration"):
    outputGraph = {}
    for sourcePID, destPIDList in forkDict.items():
        tempDestPDict = {}
        for destPID in destPIDList:
            tempDestPDict[destPID] = pDict[destPID]
            pDict[sourcePID]
            outputGraph.update(createForks([sourcePID],list(tempDestPDict.keys()),getDistances([pDict[sourcePID]],list(tempDestPDict.values()), metrics = metrics)))
    return outputGraph

In [15]:
def createProbGraph(pDict, forkDict, metrics = "duration"):
    #TODO: Kantenwerte als Wahrscheinlichkeioten...
    outputGraph = {}
    for sourcePID, destPIDList in forkDict.items():
        tempDestPDict = {}
        for destPID in destPIDList:
            tempDestPDict[destPID] = pDict[destPID]
            pDict[sourcePID]
            distances = getDistances([pDict[sourcePID]],list(tempDestPDict.values()), metrics = metrics)
            total = sum(distances[0])                         
            ws = []
            for value in distances[0]:
                ws.append(value/total)
            outputGraph.update(createForks([sourcePID],list(tempDestPDict.keys()),[ws]))
    return outputGraph

In [5]:
pDict = {0:(7.70093,51.477473), 1:(7.70993,51.477473), 2:(7.70293,51.477473),3:(7.70393,51.477473), 4:(7.70493,51.477473)}
forkDict = {0: [2,3,4], 1:[2,3,4]}

In [6]:
createDistGraph(pDict, forkDict, metrics = "distance")

{0: {2: 200.42, 3: 219.52, 4: 382.03}, 1: {2: 1189.28, 3: 1208.38, 4: 1051.76}}

In [7]:
createDistGraph(pDict, forkDict, metrics = "duration")

{0: {2: 31.98, 3: 36.56, 4: 53.76}, 1: {2: 159.09, 3: 163.68, 4: 152.16}}

In [16]:
createProbGraph(pDict, forkDict, metrics = "duration")

{0: {2: 0.2614881439084219, 3: 0.2989370400654129, 4: 0.4395748160261651},
 1: {2: 0.3349756806266187, 3: 0.344640262775567, 4: 0.32038405659781444}}